# Aiming Simulation
This simulation shows how we can determine the azimuth and altitude motor angles to aim the gun at a target point. It uses `scipy.optimize` to iteratively optimize the angles in the rotation matrices of the motors to minimize the normal distnace between the expected trajectory and the target point.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import minimize
import time

# Helper Functions

In [2]:
def rotate_vector(azimuth_angle, altitude_angle, axis_params, azimuth_params, altitude_params):
    """
    Rotate the given vector and its origin using azimuth and altitude angles,
    explicitly considering the origins of the azimuth and altitude axes.

    :param azimuth_angle: Rotation angle around the azimuth axis (in radians).
    :param altitude_angle: Rotation angle around the altitude axis (in radians).
    :param axis_params: Dictionary containing 'origin' and 'direction' for the vector.
    :param azimuth_params: Dictionary containing 'origin' and 'axis' for the azimuth motor.
    :param altitude_params: Dictionary containing 'origin' and 'axis' for the altitude motor.
    :return: Rotated origin and direction as (rotated_origin, rotated_direction).
    """
    def rodrigues_rotation_matrix(axis, angle):
        """
        Compute the rotation matrix for a given axis and angle using Rodrigues' rotation formula.

        :param axis: Axis of rotation (3D vector).
        :param angle: Angle of rotation (in radians).
        :return: 3x3 rotation matrix.
        """
        axis = axis / np.linalg.norm(axis)  # Normalize the axis
        cos_angle = np.cos(angle)
        sin_angle = np.sin(angle)
        one_minus_cos = 1 - cos_angle

        # Rodrigues' rotation formula components
        skew_symmetric = np.array([
            [0, -axis[2], axis[1]],
            [axis[2], 0, -axis[0]],
            [-axis[1], axis[0], 0]
        ])
        return cos_angle * np.eye(3) + one_minus_cos * np.outer(axis, axis) + sin_angle * skew_symmetric

    # Extract vector origin and direction
    axis_origin = axis_params['origin']
    axis_direction = axis_params['direction']

    # Step 1: Rotate around the azimuth axis
    azimuth_origin = azimuth_params['origin']
    azimuth_axis = azimuth_params['axis']

    # Translate the vector origin relative to the azimuth axis origin
    origin_relative_to_azimuth = axis_origin - azimuth_origin
    R_azimuth = rodrigues_rotation_matrix(azimuth_axis, azimuth_angle)

    # Apply azimuth rotation
    rotated_origin_azimuth = R_azimuth @ origin_relative_to_azimuth + azimuth_origin
    rotated_direction_azimuth = R_azimuth @ axis_direction

    # Step 2: Rotate around the altitude axis
    altitude_origin_relative_to_azimuth = altitude_params['origin'] - azimuth_origin
    altitude_origin = R_azimuth @ altitude_origin_relative_to_azimuth + azimuth_origin
    altitude_axis = R_azimuth @ altitude_params['axis']

    # Translate the origin relative to the altitude axis origin
    origin_relative_to_altitude = rotated_origin_azimuth - altitude_origin
    R_altitude = rodrigues_rotation_matrix(altitude_axis, altitude_angle)

    # Apply altitude rotation
    rotated_origin = R_altitude @ origin_relative_to_altitude + altitude_origin
    rotated_direction = R_altitude @ rotated_direction_azimuth

    return rotated_origin, rotated_direction


# Function to minimize (error between rotated axis and target direction)
def objective(angles, target, axis_params, azimuth_params, altitude_params):
    azimuth_angle, altitude_angle = angles
    # Rotate the axis and calculate the direction
    rotated_origin, rotated_direction = rotate_vector(azimuth_angle, altitude_angle, axis_params, azimuth_params, altitude_params)

    # Calculate the direction to the target point
    direction_to_target = target - rotated_origin
    direction_to_target = direction_to_target / np.linalg.norm(direction_to_target)  # Normalize

    # Minimize the difference between the rotated direction and target direction
    error = np.linalg.norm(rotated_direction - direction_to_target)
    return error

# Function to calculate the shortest normal distance from the vector to the target point
def calculate_shortest_distance(rotated_direction, target, rotated_origin):
    direction_to_target = target - rotated_origin
    cross_product = np.cross(direction_to_target, rotated_direction)
    distance = np.linalg.norm(cross_product) / np.linalg.norm(rotated_direction)
    return distance

# Main function to optimize the azimuth and altitude angles
def optimize_rotation(azimuth_params, altitude_params, axis_params, target):
    # Extract parameters
    axis_origin = axis_params['origin']
    axis_direction = axis_params['direction']

    azimuth_axis = azimuth_params['axis']
    altitude_axis = altitude_params['axis']

    # Initial guess for azimuth and altitude angles (in radians)
    initial_guess = [0, 0]

    # Start timing the optimization process
    start_time = time.time()

    # Minimize the objective function to find the optimal angles
    result = minimize(objective, initial_guess, args=(target, axis_params, azimuth_params, altitude_params), bounds=[(-np.pi, np.pi), (-np.pi/2, np.pi/2)])

    # End timing the optimization process
    end_time = time.time()

    # Extract the optimal angles in radians
    optimal_azimuth, optimal_altitude = result.x

    # Convert the angles to degrees for readability
    optimal_azimuth_deg = np.degrees(optimal_azimuth)
    optimal_altitude_deg = np.degrees(optimal_altitude)

    # Calculate the final vector after the optimal rotation
    final_origin, final_vector = rotate_vector(optimal_azimuth, optimal_altitude, axis_params, azimuth_params, altitude_params)

    # Calculate the error: shortest normal distance between the vector and the target
    distance = calculate_shortest_distance(final_vector, target, final_origin)

    # Print the results
    print(f"Optimal Azimuth Angle: {optimal_azimuth_deg:.2f} degrees")
    print(f"Optimal Altitude Angle: {optimal_altitude_deg:.2f} degrees")
    print(f"Final Vector: {final_vector}")
    print(f"Final Origin: {final_origin}")
    print(f"Shortest Normal Distance to Target: {distance:.3f} units")

    # Print the time taken for the optimization
    print(f"Optimization took {1000 * (end_time - start_time):.3f} ms")

    return [(optimal_azimuth, optimal_altitude),final_vector, final_origin]



def create_cylinder(origin, axis, radius, height, azimuth_angle=0, resolution=50):
    """
    Create the vertices of a rotated cylinder for plotting.
    :param origin: The base center of the cylinder (numpy array).
    :param axis: The direction of the cylinder's axis (numpy array).
    :param radius: Radius of the cylinder.
    :param height: Height of the cylinder.
    :param azimuth_angle: Rotation angle around the azimuth axis (in radians).
    :param resolution: Number of points for circle discretization.
    :return: (x, y, z) coordinates of the cylinder.
    """
    # Normalize the axis
    axis = axis / np.linalg.norm(axis)

    # Apply azimuth rotation to the cylinder's axis
    azimuth_rotation_matrix = np.array([
        [np.cos(azimuth_angle), -np.sin(azimuth_angle), 0],
        [np.sin(azimuth_angle), np.cos(azimuth_angle), 0],
        [0, 0, 1]
    ])
    rotated_axis = azimuth_rotation_matrix @ axis

    # Generate orthogonal vectors for the circular base
    if np.allclose(rotated_axis, [0, 0, 1]):
        # If axis is along z, choose x and y for base
        u = np.array([1, 0, 0])
        v = np.array([0, 1, 0])
    else:
        # Generate orthogonal vectors using cross products
        u = np.cross(rotated_axis, [0, 0, 1])
        u = u / np.linalg.norm(u)
        v = np.cross(rotated_axis, u)

    # Create a circle in the plane orthogonal to the rotated axis
    theta = np.linspace(0, 2 * np.pi, resolution)
    circle = radius * (np.outer(np.cos(theta), u) + np.outer(np.sin(theta), v))

    # Create the bottom and top circles of the cylinder
    bottom_circle = circle + origin
    top_circle = circle + origin + rotated_axis * height

    # Stack the coordinates for plotting
    x = np.vstack((bottom_circle[:, 0], top_circle[:, 0]))
    y = np.vstack((bottom_circle[:, 1], top_circle[:, 1]))
    z = np.vstack((bottom_circle[:, 2], top_circle[:, 2]))

    return x, y, z

In [3]:
def plot_components(fig, az_params, al_params, ax_params, result, color='green', plot_assembly=True, prefix=""):

  normalized_trajectory = result[1] / np.linalg.norm(result[1])
  rotated_origin = result[2]

  if plot_assembly:
    # Add the azimuth motor as a cylinder
    azimuth_cylinder = create_cylinder(
        az_params['origin'],
        az_params['axis'],
        az_params['radius'],
        az_params['length']
    )
    fig.add_trace(go.Surface(
        x=azimuth_cylinder[0],
        y=azimuth_cylinder[1],
        z=azimuth_cylinder[2],
        showscale=False,
        opacity=0.5,
        colorscale=[[0, color], [1, color]]
    ))

    # Add the altitude motor as a cylinder
    altitude_cylinder = create_cylinder(
        al_params['origin'],
        al_params['axis'],
        al_params['radius'],
        al_params['length'],
        azimuth_angle = result[0][0]
    )
    fig.add_trace(go.Surface(
        x=altitude_cylinder[0],
        y=altitude_cylinder[1],
        z=altitude_cylinder[2],
        showscale=False,
        opacity=0.5,
        colorscale=[[0, color], [1, color]]
    ))

    # Add the laser pointer as a thick line

    start = rotated_origin
    end = rotated_origin + normalized_trajectory * ax_params['length']
    fig.add_trace(go.Scatter3d(
        x=[start[0], end[0]],
        y=[start[1], end[1]],
        z=[start[2], end[2]],
        mode='lines',
        line=dict(color=color, width=10),
    ))

  # Add the trajectory as dashed blue line
  start = rotated_origin + normalized_trajectory * ax_params['length']
  end = rotated_origin + normalized_trajectory * 5000
  fig.add_trace(go.Scatter3d(
      x=[start[0], end[0]],
      y=[start[1], end[1]],
      z=[start[2], end[2]],
      mode='lines',
      line=dict(color=color, width=5, dash='dash'),
      name=prefix+"Trajectory"
  ))


# Setting Simulation Parameters
These are the parameters that the algorithm will determine from the assembly's geometry relative to the fiducials.

In [4]:
noise_magnitude_position = 1
noise_magnitude_direction = 0.01

# Uncomment to generate new noise vectors, want to use the same noise vectors to compare between this notebook and the Intersecting Axes notebook
# position_noise_vectors = np.random.normal(0, noise_magnitude_position, size=(3,3))
# direction_noise_vectors = np.random.normal(0, noise_magnitude_direction, size=(3,3))

position_noise_vectors = np.array([[-0.98465473,  0.2169746,  -0.47258125],
 [ 0.60901181,  0.3661066,  -0.56896886],
 [-0.38222116, -0.01193916,  0.09955876]])
direction_noise_vectors = np.array([[-0.02070715, -0.00824072, -0.00234403],
 [-0.0232364,  -0.00349474,  0.00894407],
 [ 0.00148924, -0.00256017,  0.00455199]])
print(position_noise_vectors)
print(direction_noise_vectors)

[[-0.98465473  0.2169746  -0.47258125]
 [ 0.60901181  0.3661066  -0.56896886]
 [-0.38222116 -0.01193916  0.09955876]]
[[-0.02070715 -0.00824072 -0.00234403]
 [-0.0232364  -0.00349474  0.00894407]
 [ 0.00148924 -0.00256017  0.00455199]]


In [5]:
# Original parameters
azimuth_params = {
    'origin': np.array([0, 0, 0], dtype=float),
    'axis': np.array([0, 0, 1], dtype=float),
    'radius': 50.0,
    'length': 100.0
}

altitude_params = {
    'origin': np.array([50, 0, 300], dtype=float),
    'axis': np.array([1, 0, 0], dtype=float),
    'radius': 50.0,
    'length': 100.0
}

axis_params = {
    'origin': np.array([0, 0, 150], dtype=float),
    'direction': np.array([0, 1, 0], dtype=float),
    'length': 150.0
}



# Adding noise to create the "actual" parameters
azimuth_params_actual = {
    'origin': azimuth_params['origin'] + position_noise_vectors[0],
    'axis': azimuth_params['axis'] + direction_noise_vectors[0],
    'radius': azimuth_params['radius'],
    'length': azimuth_params['length']
}

altitude_params_actual = {
    'origin': altitude_params['origin'] + position_noise_vectors[1],
    'axis': altitude_params['axis'] + direction_noise_vectors[1],
    'radius': altitude_params['radius'],
    'length': altitude_params['length']
}

axis_params_actual = {
    'origin': axis_params['origin'] + position_noise_vectors[2],
    'direction': axis_params['direction'] + direction_noise_vectors[2],
    'length': axis_params['length']
}

# Simulate Aiming For a Single Point
This first simple example demonstrates aiming at a single point. Note that the optimization is very quick: 5-10ms.

In [37]:
# Define the target point
target = np.array([1000, 1860, 846])

# aim straight on
# target = np.array([100, 1860, 350])

# No altitude rotation
# target = np.array([1000, 1860, 350])

# No azimuth rotation
# target = np.array([100, 1860, 1500])

# Run the optimization
result = optimize_rotation(azimuth_params, altitude_params, axis_params, target)

Optimal Azimuth Angle: -28.26 degrees
Optimal Altitude Angle: 18.44 degrees
Final Vector: [0.44922241 0.83555368 0.31630567]
Final Origin: [ 22.46726908  41.78912041 157.70140115]
Shortest Normal Distance to Target: 0.000 units
Optimization took 30.741 ms


## Ideal Scenario: Actual Axes Are the Same as Expected Axes
i.e. the axes directions and positions used in the optimization are exactly equal to their directions and positions in real-life mechanical assembly. This will likely not be the case.

In [ ]:
# Create a figure
fig = go.Figure()

In [ ]:
# Add a scatter point for the target
fig.add_trace(go.Scatter3d(
    x=[target[0]],
    y=[target[1]],
    z=[target[2]],
    mode='markers',
    marker=dict(color='red', size=5),
    name='Target'
))


plot_components(fig, azimuth_params, altitude_params, axis_params, result)
plot_components(fig, azimuth_params, altitude_params, axis_params, [(0,0),np.array([0,1,0]),axis_params['origin']], color='red')



# Update the layout for the 3D visualization
fig.update_layout(scene=dict(
    aspectmode='manual',
    aspectratio=dict(
        x=1,  # Adjust this value if your data's X dimension is significantly different
        y=1,  # Adjust this value if your data's Y dimension is significantly different
        z=1   # Increase Z ratio to avoid squishing
    ),
    xaxis=dict(range=[-3000, 3000], title='X'),
    yaxis=dict(range=[-3000, 3000], title='Y'),
    zaxis=dict(range=[0, 6000], title='Z')
), width=1000, height=800,showlegend=False)

fig.show()

## Realistic Scenario:  Actual Axes Are Not the Same as Expected Axes
As a result of small deviations in mechanical assembly and camera noise, the directions and origins of the real-life components are not exactly the same as the parameters used in the optimization. We simulate this discrepancy by adding random noise to azimuth_params_actual, altitude_params_actual and axis_params_actual.

In [ ]:
# First calculate the actual trajectory

az = result[0][0]
al = result[0][1]

origin, trajectory = rotate_vector(az, al, axis_params_actual, azimuth_params_actual, altitude_params_actual)

actual_result = [(az, al),trajectory, origin]

print(result)
print(actual_result)

[(-0.4932999329145247, 0.3218326760974571), array([0.44922241, 0.83555368, 0.31630567]), array([ 22.46726908,  41.78912041, 157.70140115])]
[(-0.4932999329145247, 0.3218326760974571), array([0.46578171, 0.8601991 , 0.30165211]), array([ 22.3942395 ,  42.45361052, 157.97011454])]


In [ ]:
# Create a figure
fig = go.Figure()

In [ ]:


# Add a scatter point for the target
fig.add_trace(go.Scatter3d(
    x=[target[0]],
    y=[target[1]],
    z=[target[2]],
    mode='markers',
    marker=dict(color='red', size=5),
    name='Target'
))


plot_components(fig, azimuth_params_actual, altitude_params_actual, axis_params_actual, actual_result, color='blue')


# Update the layout for the 3D visualization
fig.update_layout(scene=dict(
    aspectmode='manual',
    aspectratio=dict(
        x=1,
        y=1,
        z=1
    ),
    xaxis=dict(range=[-3000, 3000], title='X'),
    yaxis=dict(range=[-3000, 3000], title='Y'),
    zaxis=dict(range=[0, 6000], title='Z')
), width=1000, height=800,showlegend=False)

fig.show()

# Both Scenarios on the Same Plot

In [ ]:
# Create a figure
fig = go.Figure()

# Add a scatter point for the target
fig.add_trace(go.Scatter3d(
    x=[target[0]],
    y=[target[1]],
    z=[target[2]],
    mode='markers',
    marker=dict(color='red', size=5),
    name='Target'
))

plot_components(fig, azimuth_params, altitude_params, axis_params, result, color='green')
plot_components(fig, azimuth_params_actual, altitude_params_actual, axis_params_actual, actual_result, color='blue')


# Update the layout for the 3D visualization
fig.update_layout(scene=dict(
    aspectmode='manual',
    aspectratio=dict(
        x=1,
        y=1,
        z=1
    ),
    xaxis=dict(range=[-3000, 3000], title='X'),
    yaxis=dict(range=[-3000, 3000], title='Y'),
    zaxis=dict(range=[0, 6000], title='Z')
), width=1000, height=800,showlegend=False)

fig.show()

As you can see, the difference between the expected trajectory (green) and actual trajectory (blue) can be quite significant depending on the magnitude of the noise.

# Simulating Several Target Points

In [ ]:
# Generate random target points within specified ranges
num_points = 100  # Number of target points to generate

target_points = [
    np.array([
        np.random.uniform(-500, 500),  # x-coordinate
        np.random.uniform(500, 1500),  # y-coordinate
        np.random.uniform(300, 1500)      # z-coordinate
    ]) for _ in range(num_points)
]

target_points

[array([-266.96871544,  811.84431271, 1100.14812945]),
 array([-419.44081699,  833.36196522, 1164.16654311]),
 array([-484.61687438, 1067.86356438,  893.7809356 ]),
 array([-389.72040879,  910.99779415, 1265.50372696]),
 array([-193.27513593,  818.90566686,  845.747978  ]),
 array([ 416.85594267, 1250.57343413,  813.56388854]),
 array([ 433.95710148, 1076.52759217, 1406.53554646]),
 array([-272.92191636, 1301.86529517,  577.23594792]),
 array([-215.64590331, 1245.60462102, 1079.0177273 ]),
 array([ 464.44470172, 1449.32148105,  460.91383855]),
 array([  5.6382314 , 739.00042402, 447.11089577]),
 array([-479.63886258,  902.4527757 ,  420.47074599]),
 array([ 112.24618345, 1416.58622011,  400.87484487]),
 array([-315.81774826,  692.09138055, 1342.84634605]),
 array([-140.71049102,  922.66233397,  392.08475864]),
 array([-213.22750439, 1285.95353663,  386.92917742]),
 array([  56.3089217 , 1014.47058622,  885.14617981]),
 array([-445.57128598, 1415.77435228, 1417.96204059]),
 array([-476.

In [ ]:
predicted_results = []
actual_results = []
for target in target_points:
  # Run the optimization
  result = optimize_rotation(azimuth_params, altitude_params, axis_params, target)
  predicted_results.append(result)

    # First calculate the actual trajectory

  az = result[0][0]
  al = result[0][1]

  origin, trajectory = rotate_vector(az, al, axis_params_actual, azimuth_params_actual, altitude_params_actual)

  actual_result = [(az, al),trajectory, origin]
  actual_results.append(actual_result)

print(predicted_results)
print('------------------------------------------------------------------')
print(actual_results)

Optimal Azimuth Angle: 18.20 degrees
Optimal Altitude Angle: 50.48 degrees
Final Vector: [-0.198802    0.60455125  0.77135955]
Final Origin: [-36.14422931 109.91357822 204.5400607 ]
Shortest Normal Distance to Target: 0.000 units
Optimization took 39.201 ms
Optimal Azimuth Angle: 26.72 degrees
Optimal Altitude Angle: 49.58 degrees
Final Vector: [-0.2914905   0.5791451   0.76133057]
Final Origin: [-51.34168259 102.00772997 202.74541319]
Shortest Normal Distance to Target: 0.000 units
Optimization took 36.519 ms
Optimal Azimuth Angle: 24.41 degrees
Optimal Altitude Angle: 33.41 degrees
Final Vector: [-0.34497393  0.76015737  0.550594  ]
Final Origin: [-34.13033349  75.20691295 174.78402426]
Shortest Normal Distance to Target: 0.000 units
Optimization took 21.687 ms
Optimal Azimuth Angle: 23.16 degrees
Optimal Altitude Angle: 50.48 degrees
Final Vector: [-0.25027571  0.58503637  0.77142369]
Final Origin: [-45.51201849 106.38741832 204.55172397]
Shortest Normal Distance to Target: 0.000 un

In [ ]:
# Create a figure
fig = go.Figure()

for target, predicted_result, actual_result in zip(target_points, predicted_results, actual_results):

  # Add a scatter point for the target
  fig.add_trace(go.Scatter3d(
      x=[target[0]],
      y=[target[1]],
      z=[target[2]],
      mode='markers',
      marker=dict(color='red', size=5),
      name='Target'
  ))

  plot_components(fig, azimuth_params, altitude_params, axis_params, predicted_result, color='green', plot_assembly=False)
  plot_components(fig, azimuth_params_actual, altitude_params_actual, axis_params_actual, actual_result, color='blue', plot_assembly=False)


# Update the layout for the 3D visualization
fig.update_layout(scene=dict(
    aspectmode='manual',
    aspectratio=dict(
        x=1,
        y=1,
        z=1
    ),
    xaxis=dict(range=[-3000, 3000], title='X'),
    yaxis=dict(range=[-3000, 3000], title='Y'),
    zaxis=dict(range=[0, 6000], title='Z')
), width=1000, height=800,showlegend=False)

fig.show()

The motors and the gun were not plotted becuase they made the plot laggy. Next, we must calculate the errors between the predicted, actual and target. Again, green is the expected trajectory and blue is the actual trajectory.

In [ ]:
# Calculate the error: shortest normal distance between the vector and the target

max_error = 0
for target, predicted_result, actual_result in zip(target_points, predicted_results, actual_results):

  predicted_vector = predicted_result[1]
  actual_vector = actual_result[1]

  predicated_rotated_origin = predicted_result[2]
  actual_rotated_origin = actual_result[2]

  print(f"Target: {target}")

  print(f"Predicted vector: {predicted_vector}, \nActual vector: {actual_vector}")

  predicted_distance = calculate_shortest_distance(predicted_vector, target, predicated_rotated_origin)
  actual_distance = calculate_shortest_distance(actual_vector, target, actual_rotated_origin)
  max_error = max(max_error, actual_distance)

  print(f"Predicted shortest dist: {predicted_distance:.5f} mm, \nActual shortest dist: {actual_distance:.5f} mm\n")

print(max_error)

Target: [-266.96871544  811.84431271 1100.14812945]
Predicted vector: [-0.198802    0.60455125  0.77135955], 
Actual vector: [-0.20644742  0.60223006  0.76785908]
Predicted shortest dist: 0.00000 mm, 
Actual shortest dist: 10.93364 mm

Target: [-419.44081699  833.36196522 1164.16654311]
Predicted vector: [-0.2914905   0.5791451   0.76133057], 
Actual vector: [-0.29962638  0.57800381  0.75567505]
Predicted shortest dist: 0.00000 mm, 
Actual shortest dist: 14.27352 mm

Target: [-484.61687438 1067.86356438  893.7809356 ]
Predicted vector: [-0.34497393  0.76015737  0.550594  ], 
Actual vector: [-0.34963471  0.75856899  0.5451952 ]
Predicted shortest dist: 0.00001 mm, 
Actual shortest dist: 10.52425 mm

Target: [-389.72040879  910.99779415 1265.50372696]
Predicted vector: [-0.25027571  0.58503637  0.77142369], 
Actual vector: [-0.25828757  0.58342097  0.76669202]
Predicted shortest dist: 0.00000 mm, 
Actual shortest dist: 14.23795 mm

Target: [-193.27513593  818.90566686  845.747978  ]
Pred

As one can see, the error is quite significant: for a noise magnitude of 0.5 (unrealistically low) we get an error of ~40mm which is almost the diameter of our ball. What's especially interesting is if you compare the predict and actual vectors the difference betwene them is tiny! It is <=1mm for most vectors, and yet it results in a huge error near the target.

This indicates that our mechanical assembly is highly sensitive to noise: even a tiny offset somewhere would make the aim bad. Therefore after performing this initial optimization we need to put a laser pointer in the barrel and do a second round of optimization based on real life measurements.

One way of doing this could be to aim the gun at the target with the predicted angles. We will see the laser dot slightly off-target. We can then manually adjust the motor angles to point the laser at the target. These manually adjusted angles can be used to recalculate the directions and positions of the motor axes and gun axis.

Or maybe we can do this manual step first, get the accurate values for actual axes and then only use the `scipy.optimize` during the actual game.

# Ideal (Intersecting) Axes Test

The next little bit serves to compare the speed and error of an ideal axes setup to the above.